In [1]:
import os 
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"

In [2]:
import requests
from requests.auth import HTTPBasicAuth
import json

from django import http
from dal import autocomplete
from django.conf import settings
#from .utils import BibsonomyEntry





def query_zotero(q, conf, page_size=20, offset=0):
    # TODO: Implement possibility that user puts in a full url referencing a specific item. Similiar to logic in `query_bibsonomy` above.
    if 'group' in conf.keys():
        add = f"groups/{conf['group']}"
    else:
        add = f"users/{conf['user']}"
    url = f'https://api.zotero.org/{add}/items'
    headers = {'Zotero-API-Key': conf['API key'], 'Zotero-API-Version': '3'}
    params = {'q': q, 'qmode': 'titleCreatorYear', 'include': 'bib', 'start': offset, 'limit': page_size}
    res = requests.get(url, headers=headers, params=params)
    return res


class BibsonomyAutocomplete(autocomplete.Select2ListView):

    def get(self, request, *args, **kwargs):
        choices = []
        offset = (int(self.request.GET.get('page', 1))-1)*self.page_size
        more = False
        q = self.request.GET.get('q')
        if len(self.q) < 3:
            choices = []
        else:
            for idx, c in enumerate(self.conf):
                if c['type'] == 'bibsonomy':
                    r = query_bibsonomy(q, c, self.page_size, offset)
                    res = r.json()
                    more = res['posts'].get('next', False)
                    if 'post' in res['posts'].keys():
                        for r in res['posts']['post']:
                            ent = BibsonomyEntry(bib_entry={'post': r})
                            choices.append({'id': ent.bib_hash, 'text': ent.autocomplete})
                elif c['type'] == 'zotero':
                    res = query_zotero(q, c, self.page_size, offset)
                    if int(res.headers['Total-Results']) > (self.page_size + offset):
                        more = True
                    for r in res.json():
                        choices.append({'id': r['links']['self']['href'], 'text': r['bib']})
        return http.HttpResponse(json.dumps({
            'results': choices + [],
            'pagination': {'more': more}
        }), content_type='application/json')

    def __init__(self, page_size=20, group=None, user=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.page_size = page_size
        self.conf = getattr(settings, "APIS_BIBSONOMY", None)
        #if 'group' in conf.keys():
        #    self.group = conf['group']
        #else:
        #    self.group = None
        #if 'user' in conf.keys():
        #    self.user = conf['user']
        #else:
        #    raise ValueError('You need to specify a User.')
        #if 'API key' in conf.keys():
        #    self.api_key = conf['API key']
        #else:
        #    raise ValueError('You need to specify a API key to access the server.')
       # if 'url' in conf.keys():
         #   self.url = conf['url']
        #else:
         #   raise ValueError('You need to specify a BASE url of the Bibsonomy instance.')

In [3]:
conf = getattr(settings, "APIS_BIBSONOMY", None)


In [4]:
conf

[{'type': 'zotero',
  'url': 'https://zotero.org/',
  'user': '11264507',
  'API key': 'Xz8HVCMcR7NNwP6uXmXvKL1Y',
  'group': '4980632'}]

In [5]:
res = query_zotero("Schematismus", conf[0])

In [6]:
res.text

'[\n    {\n        "key": "UXV5XZUM",\n        "version": 3839,\n        "library": {\n            "type": "group",\n            "id": 4980632,\n            "name": "viecpro",\n            "links": {\n                "alternate": {\n                    "href": "https://www.zotero.org/groups/4980632",\n                    "type": "text/html"\n                }\n            }\n        },\n        "links": {\n            "self": {\n                "href": "https://api.zotero.org/groups/4980632/items/UXV5XZUM",\n                "type": "application/json"\n            },\n            "alternate": {\n                "href": "https://www.zotero.org/groups/4980632/items/UXV5XZUM",\n                "type": "text/html"\n            }\n        },\n        "meta": {\n            "createdByUser": {\n                "id": 11264507,\n                "username": "viecpro_test",\n                "name": "",\n                "links": {\n                    "alternate": {\n                        "href":